8. 

In [ ]:
import TensorCrossInterpolation as TCI

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    term1 = cos(2*π*(j*k/(N*N2))) * sin(2*π*(j*l/(N*N2))) * exp(-j/N)
    term2 = sin(2*π*(j*k/(N*N2))) * cos(2*π*(j*l/(N*N2))) * exp(-j^2/(N^2))

    return term1 + term2
end


function get_tci_val(j)
    localdims = fill(N2, 2)
    tolerance = 1e-8

    g(v) = f([j, v[1], v[2]]) #the method of fix one variable
    
    tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = tolerance )
    
    mps = tci_2d.sitetensors
    
    sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
    sumL = dropdims(sum(mps[2],dims=2),dims=2)

    resultMatrix = sumK * sumL #1x1 matrix
    
    return resultMatrix[1]

end


function mcmc_tci_sum(M)
    #define sum value
    sumSgn = 0.0

    #set init point
    j = 100 #rand(1:N)
    now = get_tci_val(j)
    memJ[j] = now
    vstJ[j] = true
    sumSgn += sign(now)
    #println("sumSgn:$sumSgn")
    

    for idx in 1:(M-1)
        jump = rand(-30:30)
        #jump = rand(-round(Int,N/100):round(Int,N/100))
        
        if (j + jump) < 1 || (j + jump) > N
            sumSgn += sign(now)
            continue
        else
            j = j + jump

            #calculate proposal point
            prp = 0.0
            if vstJ[j] == false
                prp = get_tci_val(j)
                memJ[j] = prp
                vstJ[j] = true
            else
                prp = memJ[j]
            end

            #MCMC
            accRatio = abs(prp/now)
            if rand() < accRatio
                #acc
                now = prp
                sumSgn += sign(now)

            else
                #rej
                sumSgn += sign(now) 
                j = j - jump #undo
            end
        end
        #print("$(round(Int,sign(now))) ")
        print("$j ")
    end

    return sumSgn / M
end

function exact_abs_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N2
            for l in 1:N2
                sum += abs(f([j, k, l]))
            end
        end
    end
    return sum / (N*(N2^2))
end

function exact_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N2
            for l in 1:N2
                sum += f([j, k, l])
            end
        end
    end
    return sum / (N*(N2^2))
end
N = 100 #Num of j elements
N2 = 100 #Num of k,l elements
MLst = [100] #number of sampling
memJ = zeros(N)
vstJ = falses(N)

Z = exact_abs_sum()
extVal = exact_sum()
println("Z : $Z")
println("exact result : $extVal")

#Z = 0.4342940004142195

 for M in MLst
    println("====M : $M====")
    epoch = 10
    acc = 0
    accSq = 0
    for i in 1:epoch
        mcmcTciSum = Z * mcmc_tci_sum(M)
        println(mcmcTciSum)
        acc += mcmcTciSum
        accSq += mcmcTciSum^2
    end

    errVal = sqrt((accSq / epoch - (acc / epoch)^2)/epoch)

    println("MCMC-TCI result : $(acc/epoch)")
    println("errVal : $errVal")

    if extVal - (errVal) <  (acc/epoch) && (acc/epoch) < extVal + (errVal)
        println("Success :)")
    else
        println("Fail :(")
    end

end

Z : 0.4342940004142195
exact result : 0.1956521400764422
====M : 100====
77 50 58 66 56 76 57 30 18 36 41 24 24 24 24 10 39 39 17 17 28 28 28 28 27 13 16 12 11 23 35 13 16 31 11 31 35 44 44 19 19 22 7 37 18 18 22 11 10 27 27 48 52 52 30 25 25 22 6 33 12 14 19 21 42 42 42 64 45 24 10 9 30 15 15 15 15 3 1 0.34743520033137565
100 98 91 69 69 44 40 44 44 55 45 33 63 56 38 38 46 73 68 38 39 56 82 66 62 32 32 32 32 32 38 38 29 23 33 27 5 26 26 26 26 23 33 7 25 18 37 37 16 24 24 32 23 36 23 34 15 7 27 15 14 12 29 10 24 24 22 14 14 14 36 36 21 9 26 16 13 17 5 4 13 4 0.312691680298238
71 72 88 86 86 64 37 10 13 6 29 29 42 59 73 51 59 31 12 20 20 28 28 28 30 30 24 24 24 24 24 28 28 28 28 28 28 28 34 34 35 35 15 41 41 41 28 28 28 28 45 24 24 17 20 20 45 25 12 11 11 9 38 38 38 62 60 64 64 60 54 54 26 35 35 10 10 14 15 36 0.2779481602651005
97 84 66 66 66 78 78 60 66 66 66 77 77 52 25 12 12 5 6 27 36 12 9 22 26 32 24 34 13 23 36 36 35 10 7 30 30 22 30 17 17 25 32 5 17 17 12 28 28 28 20 17 17 17 34 

exact Value : 4.98699664371658e-17
==== mc step : 100====
(97 1) (98 0) (98 -1) (97 1) (98 1) (99 -1) (98 -1) (97 -1) (96 -1) (95 -1) (94 0) (94 0) (94 -1) (93 -1) (92 0) (92 0) (92 1) (93 0) (93 -1) (92 1) (93 0) (93 1) (94 -1) (93 0) (93 0) (93 0) (93 1) (94 1) (95 -1) (94 1) (95 -1) (94 1) (95 0) (95 1) (96 -1) (95 0) (95 1) (96 1) (97 1) (98 -1) (97 0) (97 -1) (96 0) (96 1) (97 -1) (96 -1) (95 -1) (94 0) (94 0) (94 -1) (93 0) (93 1) (94 1) (95 1) (96 0) (96 -1) (95 0) (95 -1) (94 0) (94 -1) (93 0) (93 0) (93 1) (94 1) (94 0) (94 -1) (93 1) (94 0) (94 1) (95 1) (96 1) (96 -1) (95 0) (95 0) (95 1) (96 -1) (95 1) (95 0) (95 1) (96 0) (96 1) (97 0) (97 1) (97 1) (98 0) (98 -1) (97 1) (97 0) (97 1) (98 1) (99 -1) (98 1) (99 1) (99 1) (99 -1) (98 -1) (97 1) (97 0) (97 -1) (40 1) (41 0) (41 -1) (40 -1) (39 0) (39 -1) (38 0) (38 1) (39 0) (39 1) (40 0) (40 0) (40 0) (40 1) (41 -1) (40 -1) (39 1) (40 1) (41 1) (42 -1) (41 1) (41 1) (42 0) (42 -1) (41 1) (42 1) (43 0) (43 0) (43 1) (44 0) (4

exact Value : 4.98699664371658e-17
==== mc step : 100====
MC result : 0.006480517261977348
errVal : 0.020185128122653093
Success :)
==== mc step : 1000====
MC result : -0.0003464317353771309
errVal : 0.004946115765609209
Success :)
==== mc step : 10000====
MC result : -0.0006790518098962015
errVal : 0.00141465338239357
Success :)
==== mc step : 100000====
MC result : 0.00037764831766871475
errVal : 0.0004909880615338226
Success :)
==== mc step : 1000000====
MC result : 0.00011803470537451921
errVal : 0.00023708260743964595
Success :)
